In [2]:
import os
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

# Merging datasets

In [ ]:
PATH = "./2023_DATA/datasets"
building = "LG도서관"

file = "/Users/minsk/2023ict/2023_DATA/datasets/LG도서관/2022-07-01.csv"
lg = pd.read_csv(file)

In [ ]:
lg.head()

In [ ]:
buildings = list(map(lambda x: x.split("/")[-1], glob.glob(PATH+"/*")))
buildings

In [ ]:
for b in tqdm(buildings):
    
    extension = 'csv'
    all_filenames = [i for i in glob.glob('./2023_DATA/datasets/{}/*.{}'.format(b,extension))]
    
    #combine all files in the list
    combined_csv = pd.concat([pd.read_csv(f) for f in sorted(all_filenames) ])
    #export to csv
    stored = os.path.join(PATH, b)
    combined_csv.to_csv(stored+"/combined.csv", index=False)

In [ ]:
PATH_original = "/Users/minsk/MySolar/data_combined/"

In [ ]:
for building in tqdm(buildings):
    new_data_path = PATH+"/{}/combined.csv".format(building)
    prev_data_path = PATH_original+"/{}/combined.csv".format(building)
    
    prev = pd.read_csv(prev_data_path)
    new = pd.read_csv(new_data_path)
    
    whole = pd.concat([prev, new], ignore_index=True)
    whole = whole.drop_duplicates(ignore_index=True)
    whole.to_csv("./datasets/{}-daily-pv.csv".format(building), index=False)
    

# EDA & Preprocessing

In [3]:
PATH = "/Users/minsk/2023ict/datasets"

whole = pd.read_csv(glob.glob(PATH+"/*.csv")[0])

In [44]:
glob.glob(PATH+"/*.csv")[0].split("/")[-1].split("-")[0]

'산업협력관'

In [4]:
whole.columns

Index(['누적발전량(kWh)', '시간당발전량(kWh)', '수평면(w/㎡)', '외기온도(℃)', '경사면(w/㎡)',
       '모듈온도(℃)', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '일조(hr)', '일사(MJ/m2)', '적설(cm)',
       '전운량(10분위)', '시정(10m)', '지면온도(°C)'],
      dtype='object')

In [5]:
# drop unnecessary columns; irrelevants, duplicates, too much NaN/0.0 values, etc.
whole = whole.drop(columns=['누적발전량(kWh)', '적설(cm)', '풍속(m/s)', '풍향(16방위)', '외기온도(℃)', '강수량(mm)', '지면온도(°C)' ])

In [ ]:
for col in whole.columns:
    print(whole[col].describe())

## Fillning NaN

In [7]:
whole.isnull().sum()

시간당발전량(kWh)      0
수평면(w/㎡)         0
경사면(w/㎡)         0
모듈온도(℃)          0
일시               0
기온(°C)           8
습도(%)            4
증기압(hPa)         6
이슬점온도(°C)        7
현지기압(hPa)       31
일조(hr)         378
일사(MJ/m2)      377
전운량(10분위)       18
시정(10m)          2
dtype: int64

In [7]:
whole[whole['일조(hr)'].isnull()]

,시간당발전량(kWh),수평면(w/㎡),경사면(w/㎡),모듈온도(℃),일시,기온(°C),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),시정(10m)
17184,0.0,0.0,0.0,23.0,2022-07-01 00:00:00,25.6,99.0,32.4,25.4,1006.1,NaN,NaN,3.0,2000.0
17185,0.0,0.0,0.0,22.9,2022-07-01 01:00:00,25.1,99.0,31.5,24.9,1006.1,NaN,NaN,2.0,2000.0
17186,0.0,0.0,0.0,22.8,2022-07-01 02:00:00,25.0,99.0,31.3,24.8,1005.8,NaN,NaN,4.0,2000.0
17187,0.0,0.0,0.0,20.8,2022-07-01 03:00:00,24.9,99.0,31.1,24.7,1006.0,NaN,NaN,2.0,2000.0
17188,0.0,0.0,0.0,20.2,2022-07-01 04:00:00,24.5,99.0,30.4,24.3,1006.2,NaN,NaN,3.0,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18650,0.0,0.0,0.0,22.8,2022-08-31 02:00:00,23.5,99.0,28.6,23.3,998.8,NaN,NaN,10.0,147.0
18651,0.0,0.0,0.0,22.8,2022-08-31 03:00:00,23.4,99.0,28.4,23.2,998.4,NaN,NaN,9.0,271.0
18652,0.0,0.0,0.0,20.3,2022-08-31 04:00:00,23.1,99.0,27.9,22.9,998.3,NaN,NaN,10.0,702.0
18653,0.0,0.0,0.0,20.2,2022-08-31 05:00:00,22.6,99.0,27.1,22.4,998.6,NaN,NaN,10.0,1764.0


In [8]:
whole['TIME'] = pd.to_datetime(whole['일시']).dt.hour
whole['TIME'].describe()

count    18672.000000
mean        11.500000
std          6.922372
min          0.000000
25%          5.750000
50%         11.500000
75%         17.250000
max         23.000000
Name: TIME, dtype: float64

In [ ]:
for hour in range(7, 12):
    print(whole[['일조(hr)', 'TIME']][whole['일조(hr)'].isnull()][whole['TIME'] == hour])

In [ ]:
for hour in range(12, 20):
    print(whole[['일조(hr)', 'TIME']][whole['일조(hr)'].isnull()][whole['TIME'] == hour])

In [ ]:
print(whole[['일조(hr)', 'TIME']][whole['일조(hr)'].isnull()][whole['TIME'] == 3.0])

In [9]:
whole['일조(hr)']=whole['일조(hr)'].fillna(0.0) # 

In [10]:
whole['일사(MJ/m2)']=whole['일사(MJ/m2)'].fillna(0.0)

In [11]:
whole['전운량(10분위)']=whole['전운량(10분위)'].fillna(0.0)

In [ ]:
for i in whole[whole['현지기압(hPa)'].isnull()].index:
    whole['현지기압(hPa)'].iloc[i] = whole['현지기압(hPa)'].iloc[i-24].copy()

In [13]:
#10698
whole['현지기압(hPa)'].iloc[10698+72]

1005.9

In [14]:
whole['일시']=pd.to_datetime(whole['일시'])

In [15]:
whole["MONTH"] = pd.to_datetime(whole['일시']).dt.month_name

In [16]:
whole[['시간당발전량(kWh)', 'TIME', '일시']][whole['시간당발전량(kWh)']=='-'].index

Index([305, 306, 307, 308, 309, 310, 311], dtype='int64')

In [17]:
for i in whole[whole['시간당발전량(kWh)']=='-'].index:
    whole['시간당발전량(kWh)'].iloc[i] = whole['시간당발전량(kWh)'].iloc[i-24]

/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/3033965572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  whole['시간당발전량(kWh)'].iloc[i] = whole['시간당발전량(kWh)'].iloc[i-24]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/3033965572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  whole['시간당발전량(kWh)'].iloc[i] = whole['시간당발전량(kWh)'].iloc[i-24]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/3033965572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [18]:
whole['시간당발전량(kWh)']=whole['시간당발전량(kWh)'].astype('float64')

In [19]:
whole['시간당발전량(kWh)'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: 시간당발전량(kWh), dtype: float64

In [20]:
whole[['기온(°C)', '일시', 'TIME']][whole['기온(°C)'].isnull()]

,기온(°C),일시,TIME
1152,NaN,2020-08-26 00:00:00,0
1155,NaN,2020-08-26 03:00:00,3
1162,NaN,2020-08-26 10:00:00,10
1170,NaN,2020-08-26 18:00:00,18
1172,NaN,2020-08-26 20:00:00,20
10698,NaN,2021-09-29 18:00:00,18
10699,NaN,2021-09-29 19:00:00,19
10700,NaN,2021-09-29 20:00:00,20


In [21]:
whole['기온(°C)']=whole['기온(°C)'].bfill()

In [ ]:
whole[['이슬점온도(°C)', '일시', 'TIME']][whole['이슬점온도(°C)'].isnull()]

In [23]:
whole['이슬점온도(°C)']=whole['이슬점온도(°C)'].ffill() 

In [ ]:
whole[['습도(%)', '일시', 'TIME']][whole['습도(%)'].isnull()] #'이슬점온도(°C)', '습도(%)', '증기압(hPa)', '시정(10m)'

In [24]:
whole['습도(%)']=whole['습도(%)'].ffill()

In [ ]:
whole[['증기압(hPa)', '일시', 'TIME']][whole['증기압(hPa)'].isnull()]

In [25]:
whole['증기압(hPa)']=whole['증기압(hPa)'].ffill()

In [ ]:
whole[['시정(10m)', '일시', 'TIME']][whole['시정(10m)'].isnull()]

In [26]:
whole['시정(10m)']=whole['시정(10m)'].ffill()

In [27]:
whole.describe()

,시간당발전량(kWh),수평면(w/㎡),경사면(w/㎡),모듈온도(℃),일시,기온(°C),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),시정(10m),TIME
count,18672.000000,18672.000000,18672.000000,18672.000000,18672,18672.000000,18672.000000,18672.000000,18672.000000,18672.000000,18672.000000,18672.000000,18672.000000,18672.000000,18672.000000
mean,12.184067,159.082851,173.260872,17.412420,2021-08-04 11:13:20.514139136,15.676259,72.401484,15.568691,9.929043,1007.753310,0.253037,0.637057,5.648297,1606.765638,11.500000
min,-83.700000,0.000000,0.000000,-2566.500000,2020-07-09 00:00:00,-13.400000,8.000000,0.600000,-28.800000,975.600000,0.000000,0.000000,0.000000,10.000000,0.000000
25%,0.000000,0.000000,0.000000,5.300000,2021-01-21 11:45:00,7.700000,55.000000,6.200000,0.200000,1001.300000,0.000000,0.000000,2.000000,1274.750000,5.750000
50%,0.000000,0.000000,0.000000,18.400000,2021-08-03 23:30:00,17.200000,75.000000,12.200000,9.900000,1007.600000,0.000000,0.080000,7.000000,1952.000000,11.500000
75%,15.400000,241.000000,252.000000,25.800000,2022-02-16 11:15:00,24.100000,95.000000,25.000000,21.100000,1014.200000,0.500000,1.070000,9.000000,2000.000000,17.250000
max,110.800000,1300.000000,16849.000000,63.300000,2022-08-31 23:00:00,34.500000,99.000000,41.900000,29.800000,1027.500000,1.000000,3.880000,10.000000,12000.000000,23.000000
std,24.572332,251.707443,300.181629,24.207707,NaN,9.853137,22.703060,10.452213,11.795735,8.104687,0.396782,0.925377,3.819079,553.458427,6.922372


In [28]:
whole.isnull().sum()

시간당발전량(kWh)    0
수평면(w/㎡)       0
경사면(w/㎡)       0
모듈온도(℃)        0
일시             0
기온(°C)         0
습도(%)          0
증기압(hPa)       0
이슬점온도(°C)      0
현지기압(hPa)      0
일조(hr)         0
일사(MJ/m2)      0
전운량(10분위)      0
시정(10m)        0
TIME           0
MONTH          0
dtype: int64

In [29]:
def plot_density_hist(data: pd.DataFrame, cols: pd.Index, bins = 10):
    for col in cols:
        sns.set_style("white")
        sns.displot(data[col], bins = bins, rug=True, kind="hist")
        plt.title('Histogram of ' + col) # Give the plot a main title
        plt.xlabel(col) # Set text for the x axis
        plt.ylabel('Number of data')# Set text for y axis
        plt.show()

In [ ]:
plot_density_hist(whole.drop(columns=['일시', '시간당발전량(kWh)', 'MONTH', 'TIME']), whole.drop(columns=['일시', '시간당발전량(kWh)', 'MONTH', 'TIME']).columns)

In [38]:
whole['경사면(w/㎡)'].iloc[12300]=whole['경사면(w/㎡)'].iloc[12300-1]

/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/3879419942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  whole['경사면(w/㎡)'].iloc[12300]=whole['경사면(w/㎡)'].iloc[12300-1]


```
======================== ABNORMAL VALUES for '모듈온도(℃)' ====================

LG도서관: [13212]
기숙사B동: [12300]
다산빌딩: [13212]
대학C동: [13212]
동물실험동: [13212]
산업협력관: [12300]
삼성환경동: [13212]
시설관리동: [13212]
신재생에너지동: [13212]
중앙도서관: [13212]
중앙창고: [13212]
축구장: [13212]
학사과정: [13212]
학생회관: [13212]


======================== ABNORMAL VALUES for '경사면(w/㎡)' ====================

LG도서관: [13212]
기숙사B동: [12300]
다산빌딩: [13212]
대학C동: [13212]
동물실험동: [13212]
산업협력관: [12300]
삼성환경동: [13212]
시설관리동: [13212]
신재생에너지동: [13212]
중앙도서관: [13212]
중앙창고: [13212]
축구장: [13212]
학사과정: [13212]
학생회관: [13212]

```

In [42]:
"""Preprocessing Function"""

def fix_weather(df: pd.DataFrame, to_remove: pd.Index) -> pd.DataFrame:
    # Remove unnecessary columns
    df = df.drop(columns=to_remove)
    for col in ['이슬점온도(°C)', '습도(%)', '증기압(hPa)', '시정(10m)']:
        ## ffill()
        df[col] = df[col].ffill()
        
    df['기온(°C)'] = df['기온(°C)'].bfill() # bfill()
    
    for col in ['일조(hr)', '일사(MJ/m2)', '전운량(10분위)']:
        # fillna(0.0)
        df[col] = df[col].fillna(0.0)
    
   
    for i in df[df['현지기압(hPa)'].isnull()].index:
        # the day before
        df['현지기압(hPa)'].iloc[i] = df['현지기압(hPa)'].iloc[i-24].copy()
        
    return df


In [50]:
def fix_pvdata(df: pd.DataFrame) -> pd.DataFrame:
    # Remove culmulative data
    df = df.drop(columns=['누적발전량(kWh)'])
    
    # datatype
    for i in df[df['시간당발전량(kWh)']=='-'].index:
        # the day before
        df['시간당발전량(kWh)'].iloc[i] = df['시간당발전량(kWh)'].iloc[i-24]
        
    df['시간당발전량(kWh)'] = df['시간당발전량(kWh)'].astype('float64')
    
    # create hour and month for categorical data
    df['일시'] = pd.to_datetime(df['일시'])
    df['HOUR'] = df['일시'].dt.hour
    df['MONTH'] = df['일시'].dt.month
    
    # fix '경사면(w/㎡)'
    abnormals = df[(df['경사면(w/㎡)'] > 2500)].index
    print(abnormals)
    for i in abnormals:
        df['경사면(w/㎡)'].iloc[i]=df['경사면(w/㎡)'].iloc[i-1]
    
    #fix '모듈온도(℃)'
    abnormals = df[(df['모듈온도(℃)'] < -500)].index
    print(abnormals)
    for i in abnormals:
        df['모듈온도(℃)'].iloc[i]=df['모듈온도(℃)'].iloc[i-1]
    
    return df

In [53]:
for file in tqdm(glob.glob(PATH+"/*.csv")):
    data = pd.read_csv(file)
    building = file.split("/")[-1].split("-")[0]
    
    data = fix_pvdata(data)
    data = fix_weather(data, pd.Index(['적설(cm)', '풍속(m/s)', '풍향(16방위)', '외기온도(℃)', '강수량(mm)', '지면온도(°C)']))
    
    nan = data.isnull().sum().sum()
    print(f"{building} has been preprocessed; NaN: {nan}")
    data.to_csv("./train/{}.csv".format(building))
    

  0%|          | 0/14 [00:00<?, ?it/s]/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시간당발전량(kWh)'].iloc[i] = df['시간당발전량(kWh)'].iloc[i-24]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시간당발전량(kWh)'].iloc[i] = df['시간당발전량(kWh)'].iloc[i-24]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

Index([12300], dtype='int64')
Index([12300], dtype='int64')
산업협력관 has been preprocessed; NaN: 0


  7%|▋         | 1/14 [00:00<00:07,  1.76it/s]/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['경사면(w/㎡)'].iloc[i]=df['경사면(w/㎡)'].iloc[i-1]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['모듈온도(℃)'].iloc[i]=df['모듈온도(℃)'].iloc[i-1]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/1731517968.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

Index([13212], dtype='int64')
Index([13212], dtype='int64')
중앙도서관 has been preprocessed; NaN: 0


 14%|█▍        | 2/14 [00:00<00:05,  2.24it/s]/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['경사면(w/㎡)'].iloc[i]=df['경사면(w/㎡)'].iloc[i-1]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['모듈온도(℃)'].iloc[i]=df['모듈온도(℃)'].iloc[i-1]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/1731517968.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

...

 86%|████████▌ | 12/14 [00:05<00:01,  1.21it/s]/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['경사면(w/㎡)'].iloc[i]=df['경사면(w/㎡)'].iloc[i-1]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['모듈온도(℃)'].iloc[i]=df['모듈온도(℃)'].iloc[i-1]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/1731517968.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

Index([13212], dtype='int64')
Index([13212], dtype='int64')
축구장 has been preprocessed; NaN: 0


 93%|█████████▎| 13/14 [00:06<00:00,  1.49it/s]/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['경사면(w/㎡)'].iloc[i]=df['경사면(w/㎡)'].iloc[i-1]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/4255595668.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['모듈온도(℃)'].iloc[i]=df['모듈온도(℃)'].iloc[i-1]
/var/folders/5m/_dn3q76s5zb9tr9xkr_8vtnh0000gn/T/ipykernel_6191/1731517968.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

Index([13212], dtype='int64')
Index([13212], dtype='int64')
삼성환경동 has been preprocessed; NaN: 0


100%|██████████| 14/14 [00:06<00:00,  2.11it/s]
